In [1]:
# Cargar Datos CSV
import pandas as pd
data = pd.read_csv('DatosSingapore2.csv')

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Detectar columnas binarias
binary_vars = [col for col in data.columns 
               if set(data[col].dropna().unique()).issubset({0, 1})]

# 2. Detectar columnas numéricas verdaderas
numerical_vars = data.select_dtypes(include=['number']).columns
numerical_true = [col for col in numerical_vars if col not in binary_vars]

# 3. Features
X = data[binary_vars + numerical_true]

# 4. Escalar únicamente numéricas continuas
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[numerical_true] = scaler.fit_transform(X[numerical_true])

# 🔥 PASO CRÍTICO: convertir todo a float32
X_scaled = X_scaled.astype("float32")

# 5. Target
y = data["price"].astype("float32")

# 6. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Verificar NaNs en los conjuntos de datos
nan_counts = pd.DataFrame({
    "train": X_train.isna().sum(),
    "test": X_test.isna().sum()
})

print(nan_counts[nan_counts.sum(axis=1) > 0])

Empty DataFrame
Columns: [train, test]
Index: []


In [3]:
import tensorflow as tf
from tensorflow import keras

def create_model(n_neurons=64, n_layers=2, lr=0.001, dropout_rate=0.0,
                 hidden_activation="relu", output_activation="linear"):
    
    model = keras.Sequential()
    
    # Capa de entrada
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    # Capas ocultas
    for _ in range(n_layers):
        model.add(keras.layers.Dense(n_neurons, activation=hidden_activation))
        if dropout_rate > 0:
            model.add(keras.layers.Dropout(dropout_rate))

    # Capa de salida
    model.add(keras.layers.Dense(1, activation=output_activation))

    # Compilar
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"]
    )
    
    return model

In [6]:
import mlflow
import mlflow.tensorflow
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math

mlflow.set_experiment("Airbnb_price_regression")

def train_and_log(params):
    n_neurons = params["n_neurons"]
    n_layers = params["n_layers"]
    lr = params["lr"]
    dropout = params["dropout"]
    hidden_activation = params["hidden_activation"]
    output_activation = params["output_activation"]

    with mlflow.start_run():

        # Registrar parámetros del modelo
        mlflow.log_params(params)

        # Crear modelo
        model = create_model(
            n_neurons=n_neurons,
            n_layers=n_layers,
            lr=lr,
            dropout_rate=dropout,
            hidden_activation=hidden_activation,
            output_activation=output_activation
        )

        # Entrenamiento
        history = model.fit(
            X_train, y_train,
            validation_split=0.2,
            epochs=50,
            batch_size=32,
            verbose=0
        )

        # Predicción
        preds = model.predict(X_test).flatten()

        # Métricas
        mse = mean_squared_error(y_test, preds)
        rmse = math.sqrt(mse)
        mae = mean_absolute_error(y_test, preds)

        # Registrar métricas
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)

        # Registrar activaciones usadas
        mlflow.log_param("hidden_activation", hidden_activation)
        mlflow.log_param("output_activation", output_activation)

        # Guardar modelo en MLflow
        mlflow.tensorflow.log_model(model, "model")

        print(f"✔ Run logged — RMSE: {rmse:.2f}, MAE: {mae:.2f}, MSE: {mse:.2f}")

In [7]:
from itertools import product

# ============================
# Definir espacio de búsqueda
# ============================

param_grid = {
    "n_neurons":        [32, 64, 128],
    "n_layers":         [1, 2, 3],
    "lr":               [0.001, 0.0005],
    "dropout":          [0.0, 0.2],
    "hidden_activation": ["relu", "tanh", "elu"],
    "output_activation": ["linear", "relu"]
}

# Crear todas las combinaciones
search_space = []

for values in product(*param_grid.values()):
    params = dict(zip(param_grid.keys(), values))
    search_space.append(params)

print(f"Total de combinaciones: {len(search_space)}")

# ============================
# Ejecutar grid search
# ============================

for params in search_space:
    print(f"\n🚀 Ejecutando configuración: {params}")
    train_and_log(params)


Total de combinaciones: 216

🚀 Ejecutando configuración: {'n_neurons': 32, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.0, 'hidden_activation': 'relu', 'output_activation': 'linear'}
16/16 [==============================] - 0s 2ms/step


2025/11/26 18:53:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 18:53:54 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\jhoan\AppData\Local\Temp\tmpe2imhlt4\model\data\model\assets


2025/11/26 18:54:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✔ Run logged — RMSE: 44.51, MAE: 32.34, MSE: 1981.24

🚀 Ejecutando configuración: {'n_neurons': 32, 'n_layers': 1, 'lr': 0.001, 'dropout': 0.0, 'hidden_activation': 'relu', 'output_activation': 'relu'}
16/16 [==============================] - 0s 974us/step


...

✔ Run logged — RMSE: 11.44, MAE: 8.59, MSE: 130.97

🚀 Ejecutando configuración: {'n_neurons': 128, 'n_layers': 3, 'lr': 0.0005, 'dropout': 0.2, 'hidden_activation': 'elu', 'output_activation': 'relu'}
16/16 [==============================] - 0s 1ms/step


2025/11/26 19:46:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/26 19:46:59 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\jhoan\AppData\Local\Temp\tmpvjvyhoee\model\data\model\assets


2025/11/26 19:47:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


✔ Run logged — RMSE: 8.67, MAE: 6.25, MSE: 75.16
